In [46]:
from datetime import datetime, date
import pandas as pd
import requests

In [47]:
DATE_FROM = "2022-05-01"
DATE_TO = "2022-06-01"

YAKSWAP_GRAPH_URL = "https://api.thegraph.com/subgraphs/name/yieldyak/yak-aggregator"

WAVAX = "0xB31f66AA3C1e785363F0875A1B74E27b85FD66c7".lower()
USDC = "0xb97ef9ef8734c71904d8002f8b6bc66dd9c48a6e".lower()
USDCe = "0xA7D7079b0FEaD91F3e65f86E8915Cb59c1a4C664".lower()
YYAVAX = "0xF7D9281e8e363584973F946201b82ba72C965D27".lower()
DAI = "0xd586E7F844cEa2F87f50152665BCbc2C279D8d70".lower()
USDT = "0x9702230A8Ea53601f5cD2dc00fDBc13d4dF4A8c7".lower()

## Fetch & Parse data

In [48]:
def date_str_to_timestamp(_date_str_utc):
    date_object = datetime.strptime(_date_str_utc, "%Y-%m-%d")
    return date_object.timestamp()

def make_graph_query_request(_query):
    return requests.post(YAKSWAP_GRAPH_URL, json={'query': _query}).json()

def get_swaps(timestamp_from, timestamp_to):
    full_result = []
    while 1:
        print(f"querying between {timestamp_from}-{timestamp_to}")
        query = """
        {
            swapTransactions(
                orderBy: blockNumber,
                orderDirection: asc,
                where: { blockTimestamp_gte: %d, blockTimestamp_lte: %d },
                first: 1000
            ) {
                id
                blockNumber
                blockTimestamp
                swaps {
                    trader { id }
                }
                underlyingSwaps {
                    fromAmount
                    toAmount
                    fromToken { id, symbol, decimals }
                    toToken { id, symbol, decimals }
                    adapter { name }
                    logIndex
                }
            }
        }
        """ % (timestamp_from, timestamp_to)
        result = make_graph_query_request(query)["data"]["swapTransactions"]
        full_result += result
        print('found', len(result), "results")
        if len(result) < 1000:
            return full_result
        # Set timestamp gt than the most recent result
        # Note: this will skip any remaning tx in the block
        timestamp_from = int(result[-1]["blockTimestamp"])

In [49]:
tm_from = date_str_to_timestamp(DATE_FROM)
tm_to = date_str_to_timestamp(DATE_TO)
swaps_raw = get_swaps(tm_from, tm_to)

querying between 1651359600.0-1654038000.0
found 1000 results
querying between 1651493385-1654038000.0
found 1000 results
querying between 1651645973-1654038000.0
found 1000 results
querying between 1651804024-1654038000.0
found 1000 results
querying between 1651973792-1654038000.0
found 1000 results
querying between 1652089627-1654038000.0
found 1000 results
querying between 1652140817-1654038000.0
found 1000 results
querying between 1652201168-1654038000.0
found 1000 results
querying between 1652251837-1654038000.0
found 1000 results
querying between 1652276757-1654038000.0
found 1000 results
querying between 1652306492-1654038000.0
found 1000 results
querying between 1652340862-1654038000.0
found 1000 results
querying between 1652378762-1654038000.0
found 1000 results
querying between 1652461420-1654038000.0
found 1000 results
querying between 1652526632-1654038000.0
found 1000 results
querying between 1652585067-1654038000.0
found 1000 results
querying between 1652692947-1654038000

In [50]:
normalize = lambda a, d: int(a) / 10**int(d)
adapter_swaps = []

for swap_tx in swaps_raw:
    swaps = []
    blockTimestamp = pd.to_datetime(swap_tx['blockTimestamp'], unit='s')
    trader = swap_tx["swaps"][0]["trader"]["id"] if swap_tx["swaps"] else "unknown"
    blockNumber = int(swap_tx["blockNumber"])
    txHash = swap_tx["id"]
    for swap in swap_tx["underlyingSwaps"]:
        adapter_swaps.append({
            "fromAmount": normalize(swap["fromAmount"], swap["fromToken"]["decimals"]),
            "toAmount": normalize(swap["toAmount"], swap["toToken"]["decimals"]),
            "fromToken": swap["fromToken"]["id"],
            "fromTokenName": swap["fromToken"]["id"],
            "toToken": swap["toToken"]["id"],
            "toTokenName": swap["toToken"]["id"],
            "adapter": swap["adapter"]["name"],
            "blockTimestamp": blockTimestamp,
            "blockNumber": blockNumber,
            "trader": trader,
            "txHash": txHash,
            "logIndex": int(swap["logIndex"])
        })

adapter_swaps_df = pd.DataFrame(adapter_swaps)

/var/folders/8t/96s8_s3958l2jm63574ccr200000gn/T/ipykernel_74239/2553805351.py:6: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  blockTimestamp = pd.to_datetime(swap_tx['blockTimestamp'], unit='s')


In [51]:
adapter_swaps_sorted = adapter_swaps_df.sort_values(by=['txHash', 'logIndex'], ascending=True)

adapter_swaps = adapter_swaps_sorted.groupby(['txHash', 'trader']).agg(
    blockNumber=('blockNumber', 'first'),
    blockTimestamp=('blockTimestamp', 'first'),
    adapters=('adapter', lambda x: list(x.unique())),
    fromAmount=('fromAmount', 'first'),
    toAmount=('toAmount', 'last'),
    fromToken=('fromToken', 'first'),
    toToken=('toToken', 'last')
).reset_index()

adapter_swaps['tokens'] = adapter_swaps_sorted.groupby(['txHash', 'trader']).apply(
    lambda x: list(x[['fromToken']].values.ravel()) + [x['toToken'].iloc[-1]]
).reset_index(drop=True)


/var/folders/8t/96s8_s3958l2jm63574ccr200000gn/T/ipykernel_74239/3165924048.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  adapter_swaps['tokens'] = adapter_swaps_sorted.groupby(['txHash', 'trader']).apply(


In [55]:
STABLECOINS = [ USDC, USDCe, DAI, USDT ]
BASE_TOKEN = [ WAVAX, YYAVAX ] + STABLECOINS

COINGECKO_URL = "https://api.coingecko.com/api/v3/coins/"
TKN_TO_COINGECKO_ID = {}
TKN_TO_COINGECKO_ID[WAVAX] = "wrapped-avax"
TKN_TO_COINGECKO_ID[YYAVAX] = "yield-yak-avax"
DAYS_AGO = (date.today() - datetime.fromtimestamp(tm_from).date()).days

def get_tkn_prices(tkn_label):
    tkn_id = TKN_TO_COINGECKO_ID.get(tkn_label)
    if not tkn_id:
        raise Exception(f"Token label: '{tkn_label}' not supported!")
    full_url = COINGECKO_URL + tkn_id + "/market_chart"
    res = requests.get(full_url, {
        "vs_currency": "usd",
        "interval": "daily",
        "days": (DAYS_AGO + 1),
    })

    return res.json()["prices"]

prices = {}
# Fetch prices for base tokens that are not pegged to USD
for tkn in [_tkn for _tkn in BASE_TOKEN if _tkn not in STABLECOINS]:
    try:
        tkn_prices = get_tkn_prices(tkn)
        tkn_prices_df = pd.DataFrame(tkn_prices, columns=["date", "price"])
        tkn_prices_df["date"] = pd.to_datetime(tkn_prices_df["date"], unit='ms').dt.date
        tkn_prices_df.set_index("date", drop=True, inplace=True)
        # Save without duplicates
        prices[tkn] = tkn_prices_df.loc[~tkn_prices_df.index.duplicated()]
    except Exception as e:
        print(f"Error raised while fetching prices for token {tkn}", str(e))

# Return token price for a particular date
# For stablecoins always return price of 1
def get_tkn_price_for_date(date, tkn):
    print("fetching price for", tkn, date)
    return 1 if tkn in STABLECOINS else float(prices[tkn].loc[date].iloc[0])

def get_profit_val_usd(row):
    # Return null if neither token is base-token
    if row["fromToken"] in BASE_TOKEN:
        price = get_tkn_price_for_date(
            row["blockTimestamp"].date(),
            row["fromToken"]
        )
        return row["profit"] * price

## Arbs

In [56]:
yakswap_arbs = adapter_swaps.loc[lambda df: df["fromToken"] == df["toToken"]].copy()
yakswap_arbs.loc[:, "profit"] = yakswap_arbs["toAmount"] - yakswap_arbs["fromAmount"]
yakswap_arbs.loc[:, "profit_usd"] = yakswap_arbs.apply(get_profit_val_usd, axis=1)

fetching price for 0xa7d7079b0fead91f3e65f86e8915cb59c1a4c664 2022-05-10
fetching price for 0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7 2022-05-11
fetching price for 0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7 2022-05-11
fetching price for 0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7 2022-05-18
fetching price for 0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7 2022-05-22
fetching price for 0xb97ef9ef8734c71904d8002f8b6bc66dd9c48a6e 2022-05-11
fetching price for 0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7 2022-05-09
fetching price for 0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7 2022-05-13
fetching price for 0xb97ef9ef8734c71904d8002f8b6bc66dd9c48a6e 2022-05-10
fetching price for 0xb97ef9ef8734c71904d8002f8b6bc66dd9c48a6e 2022-05-15
fetching price for 0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7 2022-05-02
fetching price for 0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7 2022-05-30
fetching price for 0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7 2022-05-14
fetching price for 0xb97ef9ef8734c71904d8002f8b6bc6

KeyError: datetime.date(2022, 4, 30)

### Most profitable

In [ ]:
yakswap_arbs \
    [["txHash", "blockNumber", "trader", "fromToken", "fromAmount", "profit_usd", "profit", "adapters"]] \
    .sort_values(by="profit_usd", ascending=False) \
    .head(10)

,txHash,blockNumber,trader,fromToken,fromAmount,profit_usd,profit,adapters
5763,0x6f4f105ea04f7af228721cfa7bd6704695eacef7d4c8...,28689511,0xa8db7ed8f1e3f18d7edd5ed4d6a4edabcf9c43b0,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7,1000.0,725.187131,39.672338,"[GmxAdapterV0, CurveAtricryptoAdapterV0, Woofi..."
466,0x08862d7f630bc06246d065301d66e1714979b28c4307...,28689513,0xa8db7ed8f1e3f18d7edd5ed4d6a4edabcf9c43b0,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7,1000.0,148.648278,8.132004,"[GmxAdapterV0, CurveAtricryptoAdapterV0, Liqui..."
5139,0x632b058136ec662b4ecebd682de4352a54390bfbd447...,28278121,0xa8db7ed8f1e3f18d7edd5ed4d6a4edabcf9c43b0,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7,1000.0,128.953103,7.451829,"[GmxAdapterV0, CurveAtricryptoAdapterV0, Woofi..."
5843,0x70c6cb8cda93155fb5fdcb5b6e57f25e6450b22b576b...,27737644,0xa8db7ed8f1e3f18d7edd5ed4d6a4edabcf9c43b0,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7,1000.0,68.062928,4.058439,"[GmxAdapterV0, CurveAtricryptoAdapterV0]"
1796,0x22d64ffe1f24b61196f2a500145f399fdea561bd262f...,27742027,0xa8db7ed8f1e3f18d7edd5ed4d6a4edabcf9c43b0,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7,250.0,59.239437,3.532314,"[GmxAdapterV0, CurveAtricryptoAdapterV0, Kyber..."
5074,0x61e4621121e649694f1664d02484b35adcdc237ac534...,28838511,0xa8db7ed8f1e3f18d7edd5ed4d6a4edabcf9c43b0,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7,1000.0,50.789338,2.655744,"[LiquidityBookAdapter, LiquidityBook2Adapter]"
5709,0x6e4dcc66554f9c380341696f294b9fe7fc3a72928f52...,28624514,unknown,0x9702230a8ea53601f5cd2dc00fdbc13d4df4a8c7,2000.0,47.288799,47.288799,"[KyberElasticAdapter, CurveAtricryptoAdapterV0..."
6252,0x78c52d473cfbe6a9d6a17e1cf6e26fd522738419c506...,28624515,0xa8db7ed8f1e3f18d7edd5ed4d6a4edabcf9c43b0,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7,1000.0,45.594640,2.541304,"[GmxAdapterV0, CurveAtricryptoAdapterV0, Woofi..."
9834,0xbefbe3f7061f1760bb705a7e1a8afcdc3ae33e10ca3e...,28696680,0xa8db7ed8f1e3f18d7edd5ed4d6a4edabcf9c43b0,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7,125.0,35.315433,1.931978,"[TraderJoeYakAdapterV0, CurveAtricryptoAdapterV0]"
7358,0x8da453cf76ebb45fff65af0c60888fdf8f9f74f7387c...,28941570,0x7b3dedf3b349b8a943c18372972d150a572d2018,0xb97ef9ef8734c71904d8002f8b6bc66dd9c48a6e,2500.0,33.551279,33.551279,"[GmxAdapterV0, LiquidityBookAdapter]"


## General stats

### By traders

In [ ]:
yakswap_arbs \
    .groupby(by=["trader", "fromToken"]).agg(
        profit_usd_sum=("profit_usd", "sum"),
        profit_usd_mean=("profit_usd", "mean"),
        count=("txHash", "count")
    ) \
    .sort_values(by="profit_usd_sum", ascending=False) \
    .head(10)

profit_usd_sum  \
trader                                     fromToken                                                    
0xa8db7ed8f1e3f18d7edd5ed4d6a4edabcf9c43b0 0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7     1869.708097   
0x7b3dedf3b349b8a943c18372972d150a572d2018 0xb97ef9ef8734c71904d8002f8b6bc66dd9c48a6e      271.155335   
unknown                                    0x9702230a8ea53601f5cd2dc00fdbc13d4df4a8c7      213.200159   
0x8e3cb312afdf9bfee8337e4d3b966a2723240aec 0xa7d7079b0fead91f3e65f86e8915cb59c1a4c664      181.530154   
unknown                                    0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7      181.369246   
                                           0xb97ef9ef8734c71904d8002f8b6bc66dd9c48a6e      176.638933   
0x7b3dedf3b349b8a943c18372972d150a572d2018 0xa7d7079b0fead91f3e65f86e8915cb59c1a4c664      107.586733   
unknown                                    0xd586e7f844cea2f87f50152665bcbc2c279d8d70       63.000050   
0x7b3dedf3b349b8a943c18372972d150a572d2018 0xd586e7f844cea2f87f50152665bcbc2c279d8d70       41.726267   
0x19102320750b7d57488172b01ac0d9ec7ec77924 0xf7d9281e8e363584973f946201b82ba72c965d27       21.146793   

                                                                                       profit_usd_mean  \
trader                                     fromToken                                                     
0xa8db7ed8f1e3f18d7edd5ed4d6a4edabcf9c43b0 0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7        12.894539   
0x7b3dedf3b349b8a943c18372972d150a572d2018 0xb97ef9ef8734c71904d8002f8b6bc66dd9c48a6e         6.305938   
unknown                                    0x9702230a8ea53601f5cd2dc00fdbc13d4df4a8c7         1.349368   
0x8e3cb312afdf9bfee8337e4d3b966a2723240aec 0xa7d7079b0fead91f3e65f86e8915cb59c1a4c664         2.669561   
unknown                                    0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7         0.939737   
                                           0xb97ef9ef8734c71904d8002f8b6bc66dd9c48a6e         1.650831   
0x7b3dedf3b349b8a943c18372972d150a572d2018 0xa7d7079b0fead91f3e65f86e8915cb59c1a4c664         6.328631   
unknown                                    0xd586e7f844cea2f87f50152665bcbc2c279d8d70         2.625002   
0x7b3dedf3b349b8a943c18372972d150a572d2018 0xd586e7f844cea2f87f50152665bcbc2c279d8d70         8.345253   
0x19102320750b7d57488172b01ac0d9ec7ec77924 0xf7d9281e8e363584973f946201b82ba72c965d27         1.762233   

                                                                                       count  
trader                                     fromToken                                          
0xa8db7ed8f1e3f18d7edd5ed4d6a4edabcf9c43b0 0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7    145  
0x7b3dedf3b349b8a943c18372972d150a572d2018 0xb97ef9ef8734c71904d8002f8b6bc66dd9c48a6e     43  
unknown                                    0x9702230a8ea53601f5cd2dc00fdbc13d4df4a8c7    158  
0x8e3cb312afdf9bfee8337e4d3b966a2723240aec 0xa7d7079b0fead91f3e65f86e8915cb59c1a4c664     68  
unknown                                    0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7    193  
                                           0xb97ef9ef8734c71904d8002f8b6bc66dd9c48a6e    107  
0x7b3dedf3b349b8a943c18372972d150a572d2018 0xa7d7079b0fead91f3e65f86e8915cb59c1a4c664     17  
unknown                                    0xd586e7f844cea2f87f50152665bcbc2c279d8d70     24  
0x7b3dedf3b349b8a943c18372972d150a572d2018 0xd586e7f844cea2f87f50152665bcbc2c279d8d70      5  
0x19102320750b7d57488172b01ac0d9ec7ec77924 0xf7d9281e8e363584973f946201b82ba72c965d27     12

In [ ]:
def get_adapter_stats(arbs_df=yakswap_arbs):
    return adapter_swaps_df.merge(arbs_df, how="inner", on="txHash") \
        .groupby(by="adapter") \
        .agg(count=("txHash", "count"), profit_usd_sum=("profit_usd", "sum"), profit_usd_mean=("profit_usd", "mean")) \
        .sort_values(by="profit_usd_sum", ascending=False)

def get_hop_stats(arbs_df=yakswap_arbs):
    hop_counted = adapter_swaps_df.groupby(by="txHash").agg(hopCount=("fromAmount", "count"))
    swaps = hop_counted.merge(arbs_df, how="inner", on="txHash")
    return swaps \
        .groupby(by="hopCount") \
        .agg(count=("txHash", "count"), profit_usd_sum=("profit_usd", "sum"), profit_usd_mean=("profit_usd", "mean")) \
        .sort_values(by="profit_usd_sum", ascending=False)

def get_amountfrom_stats(arbs_df=yakswap_arbs):
    return arbs_df \
        .groupby("fromAmount") \
        .agg(count=("txHash", "count"), profit_usd_sum=("profit_usd", "sum"), profit_usd_mean=("profit_usd", "mean")) \
        .sort_values(by="profit_usd_sum", ascending=False)


def get_adapter_stats_for_trader_and_asset(trader, asset):
    arbs_trader = yakswap_arbs.loc[
        (yakswap_arbs.trader == trader) &
        (yakswap_arbs.fromToken == asset)
    ][["txHash", "profit_usd"]]
    return get_adapter_stats(arbs_trader)

def get_hop_stats_for_trader_and_asset(trader, asset):
    arbs_trader = yakswap_arbs.loc[
        (yakswap_arbs.trader == trader) &
        (yakswap_arbs.fromToken == asset)
    ][["txHash", "profit_usd"]]
    return get_hop_stats(arbs_trader)


def get_amountfrom_stats_for_trader_and_asset(trader, asset):
    arbs_trader = yakswap_arbs.loc[
        (yakswap_arbs.trader == trader) &
        (yakswap_arbs.fromToken == asset)
    ]
    return get_amountfrom_stats(arbs_trader)

### By assets

#### USDC

In [ ]:
usdc_arbs = yakswap_arbs.loc[yakswap_arbs.fromToken == USDC]

In [ ]:
get_adapter_stats(usdc_arbs)

,count,profit_usd_sum,profit_usd_mean
adapter,,,
LiquidityBookAdapter,90,306.770106,3.408557
GmxAdapterV0,44,240.624631,5.468742
PlatypusYakAdapterV2,81,161.788319,1.997387
WoofiV2Adapter,56,131.666753,2.351192
KyberElasticAdapter,52,74.810232,1.438658
CurveAtricryptoAdapterV0,16,62.806958,3.925435
LiquidityBook2Adapter,13,20.955298,1.611946
SolisnekAdapter,3,8.891428,2.963809
SynapsePlainYakAdapterV0,11,6.897274,0.627025


In [ ]:
get_hop_stats(usdc_arbs).head(5)

,count,profit_usd_sum,profit_usd_mean
hopCount,,,
2,72,311.997068,4.333293
3,78,135.797200,1.740990


In [ ]:
get_amountfrom_stats(usdc_arbs).head(5)

,count,profit_usd_sum,profit_usd_mean
fromAmount,,,
2000.0,100,167.799278,1.677993
2500.0,12,78.398379,6.533198
75000.0,4,58.995494,14.748874
10000.0,6,53.416938,8.902823
25000.0,10,50.720997,5.072100


#### WAVAX

In [ ]:
wavax_arbs = yakswap_arbs.loc[yakswap_arbs.fromToken == WAVAX]

In [ ]:
get_adapter_stats(wavax_arbs)

,count,profit_usd_sum,profit_usd_mean
adapter,,,
GmxAdapterV0,76,1643.261601,21.621863
CurveAtricryptoAdapterV0,27,1342.779038,49.732557
WoofiV2Adapter,216,1286.869126,5.957727
LiquidityBookAdapter,266,925.563650,3.479563
KyberElasticAdapter,93,230.663942,2.480257
TraderJoeYakAdapterV0,20,150.152220,7.507611
PlatypusYakAdapterV2,25,78.316033,3.132641
LiquidityBook2Adapter,4,60.400858,15.100215
GlacierAdapter,22,31.221828,1.419174


In [ ]:
get_hop_stats(wavax_arbs)

,count,profit_usd_sum,profit_usd_mean
hopCount,,,
3,114,1656.208926,14.528148
2,208,395.438089,1.901145
1,264,0.000000,0.000000


In [ ]:
get_amountfrom_stats(wavax_arbs).head(5)

,count,profit_usd_sum,profit_usd_mean
fromAmount,,,
1000.0,64,1448.719843,22.636248
250.0,36,220.942512,6.137292
125.0,45,196.752720,4.372283
100.0,176,181.369246,1.030507
2000.0,1,3.293020,3.293020
